# Targeted-IMSE applied with the change of variable in polar coordinates

In [27]:
data<-read.csv("data_visco.csv")
data_lasso <- read.csv("data_lasso.csv")
data_AIC <- read.csv("data_AIC.csv")

Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
“Fin de fichier (EOF) dans une chaîne de caractères entre guillements”


In [28]:
# We create a function that takes only the values for which the sum is under or equal to 100
Under_100 <-function(X){
    row<-vector()
    for (i in 1:dim(X)[1]){
        if (sum(X[i,-1])>100){
            row<-rbind(row,i)
        }
    }
    X<-X[-row,]
    return(X)
}

### Functions of change of variables

In [29]:
#install.packages("SphericalCubature")
library("SphericalCubature")

#### Change of  variables from a simplex to a N-sphere with the N-sphere coordinates

In [30]:
change_in_phi_from_x = function(x){ #We put in argument a matrix or a vector that takes n values in a simplex
    epsilon<-1e-9  # We had a little noise to insure that we can devide by 0
    X<-as.matrix(x[, -1]/100)
    X[which(X == NaN)] <- 0.
    for (i in 1:dim(X)[1]){   #To ensure that we each row is equal to 1 to remove the noise in the column with the highest values in it
        X[ i,23] <- X[ i,23] - (sum(X[i, ]==0.))*epsilon
    }
    X[which(X == 0.)] <- epsilon
    Y<-sqrt(X) #We do a first change of variables y²=x eq y=sqrt(x)
    Y[is.na(Y)] <- 0  #If we have some Nan in our matrix we replace them by zeros
    return(cbind(x[,1],t(as.matrix(rect2polar(t(Y))$phi))))
}

#### Change of  variables from a N-sphere to an simplex with the N-sphere coordinates

In [31]:
#This change of variable is only for a VECTOR
change_in_x_from_phi = function(phi){ #We put in argument vector that takes n-1 values of angles in [0, pi/2]
    Phi=t(phi)
    r=rep(1,dim(Phi)[1])  #To realize our change of variables considered in the function polar2rect in R it needs to take a radius equals to 1 
    new<-(t(polar2rect(r, t(Phi))))**2  # We realize the change of variables such that f.e. x_1=cos²(theta_1)
    new[which(new<1e-4)]<-0   #We obtain sometimes values of order 1e-5 we replace it by 0
    return (as.matrix(new*100))
}


### Targeted IMSE

In [32]:
library("KrigInv") #Pour t-IMSE
library("nloptr") #Pour la fonction d'optimisation Cobyla

In [54]:
#t-IMSE STANDARD APPLIQUEE AUX DONNEES 

d = dim(data_lasso)[2]
print(d)
N = 10*d
s = 0
data_Lasso_reduced = change_in_phi_from_x(data_lasso[1:N,])
data_Lasso_reduced[,1]=log(data_Lasso_reduced[,1])
shape = c(dim(data_Lasso_reduced))
print(shape)
noise = matrix(runif(prod(shape), min=0.0001, max=0.002),nrow=N)
mod = km(formula = ~1, design = data_Lasso_reduced+noise, response = data_Lasso_reduced[,1])
x = data_Lasso_reduced[1:50,]
tmse_optim(x, model = mod, T = log(400))

[1] 29


Warning message in sqrt(X):
“Production de NaN”


[1] 290  28

optimisation start
------------------
* estimation method   : MLE 
* optimisation method : BFGS 
* analytical gradient : used
* trend model : ~1
* covariance model : 
  - type :  matern5_2 
  - nugget : NO
  - parameters lower bounds :  1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 
  - parameters upper bounds :  51.00711 0.003794731 0.9899475 2.091367 0.1646847 0.003765121 0.7363443 0.1369836 0.003766656 0.003775912 0.003750972 0.05714827 1.625098 0.003776339 1.139104 0.7173561 0.2675296 0.003777361 1.681634 0.003777537 0.2751725 0.003779556 2.186736 0.003763288 1.049107 1.233751 1.573257 3.141145 
  - best initial criterion value(s) :  -519.9446 

N = 28, M = 5 machine precision = 2.22045e-16
At X0, 0 variables are exactly at the bounds
At iterate     0  f=       519.94  |proj g|=       1.5165
At iterate     1  f =       454.17  |proj g|=        1.219

[1] 0.44509246 0.44640145 0.02860443 0.02770142 0.03004244 0.14389084
 [7] 0.18694409 0.43065427 0.50107033 0.14044165 0.04407115 0.69691027
[13] 0.57482709 0.55317649 0.43820342 0.12150932 0.30973743 0.13010002
[19] 0.03392324 0.05827457 0.05827457 0.04273140 0.04273140 0.02947247
[25] 0.02499189 0.14595527 0.04170752 0.07267689 0.06115037 0.02571691
[31] 0.02571691 0.22009931 0.07826826 0.35440235 0.08730306 0.34501954
[37] 0.14195230 0.17847158 0.24578014 0.05222477 0.04886545 0.04715804
[43] 0.14863359 0.37408205 0.27439321 0.17072365 0.03145539 0.08026963
[49] 0.01881938 0.01855095

In [72]:
X = data_Lasso_reduced[,c(-1)]

In [57]:
#TROUVER LE POINT LE PLUS PROCHE D'UN NOUVEAU POINT

#input: nouveau point: 'x_new' ; donnees dans lesquelles on cherche: 'data'
#output: la valeur y du point le plus proche

pointProche <- function(x_new, data){
    X_cher = data[,c(-1)]
    argmin = 1
    minvalue = norm(x_new- X_cher[1,], type = "2") 
    n_points = nrow(data) #longeur da la base de données

    for (i in 2:n_points){
        mse = norm(x_new- X_cher[i,], type = "2")
        
        if (mse < minvalue){
            minvalue = mse 
            argmin = i
        }
    }
    
    return (data[argmin,])
}

In [77]:
noise = matrix(runif(prod(dim(X)),min = 0.0001,max = 0.0002), nrow = nrow(X))
m0 = km(formula = ~1, design = X + noise, response = data_Lasso_reduced[,1])


optimisation start
------------------
* estimation method   : MLE 
* optimisation method : BFGS 
* analytical gradient : used
* trend model : ~1
* covariance model : 
  - type :  matern5_2 
  - nugget : NO
  - parameters lower bounds :  1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 
  - parameters upper bounds :  0.0001976477 0.9867417 2.089039 0.1640188 0.0002111553 0.7346956 0.135028 0.0002083871 0.0002009023 0.0002037709 0.05558645 1.621901 0.0002205585 1.135581 0.7161305 0.2654269 0.0002205511 1.678296 0.0002173511 0.2717365 0.000209942 2.184529 0.0001977104 1.047228 1.230839 1.570729 3.141585 
  - best initial criterion value(s) :  -638.5286 

N = 27, M = 5 machine precision = 2.22045e-16
At X0, 0 variables are exactly at the bounds
At iterate     0  f=       638.53  |proj g|=       1.6067
At iterate     1  f =       507.55  |proj g|=        2.5582
At iterate     2 

In [99]:
tmse_coord= function(x_new){
    # d: dimension, d = 29
    data = data_updated
    d = shape[2]
    niter = 1000
    s = 0
    
    
    y_new = pointProche(x_new, data)
    log_vis = y_new
    newpoint = c(log_vis,x_new)
    data_new = data.frame(data)
    
    print(dim(data_new))
    
    data_new[nrow(data_new) + 1, ] = newpoint
    # data_new = rbind(data_new, as.numeric(newpoint))
    noise = matrix(runif(prod(dim(data_new)),min = 0.0001,max = 0.0002), nrow = nrow(data_new))
    #mod = km(formula = ~1, design = data_new + noise, response = data_new[1],)
    mod = update(m0, newX = data_new[,c(-1)]+noise, newy = data_new[,1],newX.alreadyExist = FALSE, cov.reestim = FALSE, trend.reestim = FALSE)
    x= data_new[1:50,]
    tmse = tmse_optim(x, model = m0, T = log(400))
    return(tmse)    
}

In [100]:


# OPTIMISATION TMSE PROCESS
x = data_Lasso_reduced[1:50,]

# Some parameters
d = shape[2] # data dimension
n_init = 10*d   # initial number of data points
n_points = nrow(data_Lasso_reduced) # total number of data points
n_adds = 2 # number of additional data points
index = sample(1:nrow(data_Lasso_reduced),n_init)


data_updated = data_Lasso_reduced

print("Starting...")
# Loop over additional data points
for (i in 1:n_adds){

    print("Adding point... ")
    data_recherche = data_updated[setdiff(1:nrow(data_Lasso_reduced),index),]
    
    x_new = cobyla(x0 = as.numeric(X[i,]), fn = tmse_coord, control = list(xtol_rel = 1e-3))$par
    

    newpoint = pointProche(x_new,data_recherche)
    data_updated[nrow(data_updated) + 1, ] = newpoint
    #data_updated = rbind(data_updated,newpoint)
    X = data_updated[,c(-1)]
}



[1] "Starting..."
[1] "Adding point... "
[1] 290  28


Warning message in matrix(value, n, p):
“la longueur des données [55] n'est pas un diviseur ni un multiple du nombre de colonnes [28]”


ERROR: Error in predict_nobias_km(object = model, newdata = as.data.frame(z), : newdata must have the same numbers of columns ( 28 ) than the experimental design ( 27 )


In [98]:
help(km)

In [18]:
help(tmse_optim)

In [84]:
dim(data_Lasso_reduced)

[1] 290  28